In [1]:
#import pandas and load data into df
import pandas as pd
import numpy as np

file = "Resources/purchase_data.csv"

purchases_df = pd.read_csv(file)
purchases_df.head(10)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
5,5,Yalae81,22,Male,81,Dreamkiss,3.61
6,6,Itheria73,36,Male,169,"Interrogator, Blood Blade of the Queen",2.18
7,7,Iskjaskst81,20,Male,162,Abyssal Shard,2.67
8,8,Undjask33,22,Male,21,Souleater,1.10
9,9,Chanosian48,35,Other / Non-Disclosed,136,Ghastly Adamantite Protector,3.58


In [2]:
#support functions
def column_formatter(format_dict, df, copy_df = False):
    """
    format_dict: a dictionary of columns mapped to format ex/ {'My Column':'${:,.2f}'}
    df: DataFrame containing the columns to be formatted
    copy_df: when True returns a new DataFrame with formatted columns
             when False (default) edits and returns the DataFrame sent as arg (df)
    """
    if copy_df:
        new_df = df.copy()
        for column, form in format_dict.items():
            new_df[column] = new_df[column].map(form.format)
        return new_df
    else:
        for column, form in format_dict.items():
            df[column] = df[column].map(form.format)
        return df

In [3]:
#Unique players in purchase data
players_df = pd.DataFrame({"Total Players": [purchases_df["SN"].nunique()]})
players_df

,Total Players
0,576


In [4]:
#purchases overview
#pull series to use in data frame
unique_items = purchases_df["Item ID"].nunique()
mean_purchase_price = purchases_df["Price"].mean()
total_purchases = len(purchases_df)
total_rev = purchases_df["Price"].sum()
#load series into new dataframe
purchase_overview_df = pd.DataFrame([[unique_items, mean_purchase_price, total_purchases, total_rev]],
                                    columns=[
                                        "Unique Items Purchases",
                                        "Average Price per Purchase",
                                        "Total Number of Purchases",
                                        "Total Revenue"
                                    ])
#format output
format_map = {"Average Price per Purchase" : "${:,.2f}", "Total Revenue" : "${:,.2f}"}
column_formatter(format_map, purchase_overview_df)

,Unique Items Purchases,Average Price per Purchase,Total Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [5]:
#Gender demographics
#group by gender
gender_group = purchases_df.groupby("Gender")
#unique count
gender_count = gender_group["SN"].nunique()
#build data frame using direct row index references
gender_breakdown_df = pd.DataFrame(
    [[gender_count.index[0], gender_count[0], (gender_count[0]/gender_count.sum()) * 100],
    [gender_count.index[1], gender_count[1], (gender_count[1]/gender_count.sum()) * 100],
    [gender_count.index[2], gender_count[2], (gender_count[2]/gender_count.sum()) * 100]],
    columns = ["Gender", "Count", "Percentage"])
#use gender as index
gender_breakdown_df = gender_breakdown_df.set_index("Gender")
#format output
format_map = {"Percentage" : "{:.2f}%"}
column_formatter(format_map, gender_breakdown_df)

,Count,Percentage
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%


In [6]:
#Purchasing Analysis by gender
#use gender_group to calculate summary data for new columns
gender_analysis_df = pd.DataFrame(gender_group["Price"].count()).rename(columns={"Price":"Purchase Count"})
gender_analysis_df["Average Purchase"] = gender_group["Price"].mean().values
gender_analysis_df["Total Purchase Value"] = gender_group["Price"].sum().values
#use column from previous dataframe to calculate avg per person
gender_analysis_df["Average Total Purchase per Person"] = gender_analysis_df.iloc[:,2] / gender_breakdown_df.iloc[:,0]
#format output
format_map = {"Average Purchase":"${:,.2f}","Total Purchase Value": "${:,.2f}","Average Total Purchase per Person":"${:,.2f}"}
column_formatter(format_map, gender_analysis_df)

,Purchase Count,Average Purchase,Total Purchase Value,Average Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [7]:
#create age bins and split data into age groups
age_bins = [0,9,14,19,24,29,34,39,1000]
age_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
age_groups = purchases_df.groupby(pd.cut(purchases_df["Age"], bins = age_bins, labels = age_labels))

In [8]:
#Age Demographics
#create new dataframe from age group object
age_demo_df = pd.DataFrame(age_groups["SN"].nunique()).rename(columns={"SN" :"Total Count"})
#add percentage columns
age_demo_df["Percentage"] = age_demo_df["Total Count"] / purchases_df["SN"].nunique() * 100
#format output
format_map = {"Percentage" : "{:.2f}%"}
column_formatter(format_map, age_demo_df)

,Total Count,Percentage
Age,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [18]:
#Age Analysis
#create new dataframe and add required columns
age_analysis_df = pd.DataFrame(age_groups["Item ID"]
                               .count()).rename(columns={"Item ID" :"Purchase Count"})
age_analysis_df["% of All Purchases"] = age_groups["Item ID"].count() / len(purchases_df) * 100
age_analysis_df["Average Purchase Price"] = age_groups["Price"].mean()
age_analysis_df["Total Purchase Value"] =  age_groups["Price"].sum()
#get a new group including grouping on SNs
age_sn_groups = purchases_df.groupby([pd.cut(purchases_df["Age"],
                                             bins = age_bins,
                                             labels = age_labels),
                                      "SN"])
#calculate avg total per person by summing each sn, grouping them into age brackets and getting the mean.
age_analysis_df["Avg Total Purchase per Person"] = age_sn_groups.sum().groupby(level = "Age")["Price"].mean()
#format output
format_map = {"% of All Purchases":"{:.2f}%","Average Purchase Price":"${:,.2f}", "Total Purchase Value":"${:,.2f}", "Avg Total Purchase per Person":"${:,.2f}"}
column_formatter(format_map, age_analysis_df)

,Purchase Count,% of All Purchases,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age,,,,,
<10,23,2.95%,$3.35,$77.13,$4.54
10-14,28,3.59%,$2.96,$82.78,$3.76
15-19,136,17.44%,$3.04,$412.89,$3.86
20-24,365,46.79%,$3.05,"$1,114.06",$4.32
25-29,101,12.95%,$2.90,$293.00,$3.81
30-34,73,9.36%,$2.93,$214.00,$4.12
35-39,41,5.26%,$3.60,$147.67,$4.76
40+,13,1.67%,$2.94,$38.24,$3.19


In [10]:
#Top Spenders
#group by SN
sn_group = purchases_df.groupby("SN")
#put the 5 with the highest sum into the data frame
top_spending_df = pd.DataFrame(sn_group["Price"].sum().sort_values(ascending = False)).rename(columns = {"Price":"Total Purchase Value"})
#Add more columns to the df
top_spending_df["Purchase Count"] = sn_group["Item ID"].count()
top_spending_df["Avg Purchase Price"] = top_spending_df.iloc[:,0] / top_spending_df.iloc[:,1]
top_spending_df = top_spending_df[["Purchase Count","Avg Purchase Price","Total Purchase Value"]]
#Add formating
format_map = {"Avg Purchase Price": "${:,.2f}", "Total Purchase Value": "${:,.2f}"}
top_spending_df_formatted = column_formatter(format_map, top_spending_df, copy_df = True)
#output
top_spending_df_formatted.head()

,Purchase Count,Avg Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [11]:
#Most Popular Item
#group
items_group = purchases_df[["Item ID", "Item Name","Price"]].groupby(["Item ID", "Item Name"])
#add to datafram
items_df = pd.DataFrame(items_group["Price"].count()).rename(columns={"Price":"Purchase Count"})
#add more colums
items_df["Item Price (avg)"] = items_group["Price"].sum() / items_group["Price"].count()
items_df["Total Purchase Value"] = items_group["Price"].sum()
#sort
items_df = items_df.sort_values(by="Purchase Count", ascending = False)
#add formatting but keep original dataframe un altered
format_map = {"Item Price (avg)": "${:,.2f}", "Total Purchase Value": "${:,.2f}"}
formatted_items_df = column_formatter(format_map, items_df, copy_df = True)
formatted_items_df.head()

,,Purchase Count,Item Price (avg),Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [12]:
#Most Profitable Items
#resort previous df by Total Purchase Value
most_profitable_df = items_df.sort_values("Total Purchase Value", ascending = False)
#format output but preserve original dataframe
formatted_most_profitable_df = column_formatter(format_map, most_profitable_df, copy_df = True)
formatted_most_profitable_df.head()

,,Purchase Count,Item Price (avg),Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80


In [27]:
#Use top_spending_df to identify demo data about top % or spenders
#make list for number of SNs in top 1%, 5% and 10%
percentiles = [int(len(top_spending_df)*.01), int(len(top_spending_df)*.05), int(len(top_spending_df)*.1)]
#list of users in each percentile
top_spender_list = [top_spending_df.head(percentiles[0]).index.tolist(),
                    top_spending_df.head(percentiles[1]).index.tolist(),
                    top_spending_df.head(percentiles[2]).index.tolist()]
#filter full dataframe by SNs in list then group by SN. add similar group for full dataset
percentile_data_list = [
    purchases_df[purchases_df["SN"].isin(top_spender_list[0])].groupby("SN"),
    purchases_df[purchases_df["SN"].isin(top_spender_list[1])].groupby("SN"),
    purchases_df[purchases_df["SN"].isin(top_spender_list[2])].groupby("SN"),
    purchases_df.groupby("SN")
]
#Make DF with row for each percentile group. Columns: avg age, avg item price

whale_analysis_df = pd.DataFrame(
    {
        "Average Age":[
            percentile_data_list[0]["Age"].unique().mean()[0],
            percentile_data_list[1]["Age"].unique().mean()[0],
            percentile_data_list[2]["Age"].unique().mean()[0],
            percentile_data_list[3]["Age"].unique().mean()[0]
        ],
        "Average Item Price":[
            percentile_data_list[0]["Price"].sum().sum() / percentile_data_list[0]["Price"].count().sum(),
            percentile_data_list[1]["Price"].sum().sum() / percentile_data_list[1]["Price"].count().sum(),
            percentile_data_list[2]["Price"].sum().sum() / percentile_data_list[2]["Price"].count().sum(),
            percentile_data_list[3]["Price"].sum().sum() / percentile_data_list[3]["Price"].count().sum()
        ],
        "Average Total spent":[
            percentile_data_list[0]["Price"].sum().mean(),
            percentile_data_list[1]["Price"].sum().mean(),
            percentile_data_list[2]["Price"].sum().mean(),
            percentile_data_list[3]["Price"].sum().mean()
        ],
    },
    index = [
        "Top 1%",
        "Top 5%",
        "Top 10%",
        "Full Dataset"
    ],
)
#format output
whale_analysis_df.index.rename("Top Spenders", inplace = True)
format_map = {"Average Age":"{:.2f}", "Average Item Price":"${:,.2f}"}
column_formatter(format_map, whale_analysis_df)
#percentile_data_list[0]["Price"].mean().sum()

,Average Age,Average Item Price,Average Total spent
Top Spenders,,,
Top 1%,22.40,$3.95,14.992000
Top 5%,22.82,$3.72,11.018214
Top 10%,22.75,$3.60,9.526316
Full Dataset,22.74,$3.05,4.131545
